# Homenet
Skapar en session "spark".
_Notera att konfigurationen är uppdaterad med spark.sql.catalogImplementation = 'hive' vilket är en förutsättning för att kunna skapa permanenta tabeller._

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Homenet").config("spark.memory.offHeap.enabled", "true").config(
    "spark.memory.offHeap.size", "10g").config('spark.sql.catalogImplementation', 'hive').getOrCreate()

Läser in json-filerna för adresser, mäklare (individuella mäklare med ev info om vilka mäklarföretag de tillhör, om något), objekt (bostadsrätter) och personer (säljare och köpare).
Filerna läses först in till var sin dataframe, dataframen konverteras sedan till en temporär view - som man sedan kan ställa SQL-frågor till.

In [9]:
adresser = spark.read.json("spark-warehouse/adresser.json", multiLine=True)
adresser.createOrReplaceTempView("adresser")

maklare = spark.read.json("spark-warehouse/maklare.json", multiLine=True)
maklare.createOrReplaceTempView("maklare")

objekt = spark.read.json("spark-warehouse/objekt.json", multiLine=True)
objekt.createOrReplaceTempView("objekt")

personer = spark.read.json("spark-warehouse/personer.json", multiLine=True)
personer.createOrReplaceTempView("personer")

Så här kan man se schemat för en dataframe (eller temp view, eller tabell, etc) 

In [10]:
objekt.printSchema()

root
 |-- adress: long (nullable = true)
 |-- antal_rum: long (nullable = true)
 |-- avslutat: string (nullable = true)
 |-- balkong: string (nullable = true)
 |-- forsaljningspris: long (nullable = true)
 |-- hiss: string (nullable = true)
 |-- id: long (nullable = true)
 |-- kopare: long (nullable = true)
 |-- maklararvode: long (nullable = true)
 |-- maklare: long (nullable = true)
 |-- manadsavgift: long (nullable = true)
 |-- saljare: long (nullable = true)
 |-- saljstart: string (nullable = true)
 |-- storlek: long (nullable = true)
 |-- utgangspris: long (nullable = true)
 |-- vaning: long (nullable = true)


Så här kan man hämta ut datarader:

In [11]:
objekt.show(2,0)

+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+
|adress|antal_rum|avslutat  |balkong|forsaljningspris|hiss|id |kopare|maklararvode|maklare|manadsavgift|saljare|saljstart |storlek|utgangspris|vaning|
+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+
|75    |6        |2020-07-19|1      |14561000        |1   |1  |5     |218415      |2      |3090        |107    |2020-01-25|137    |12420000   |5     |
|72    |5        |2020-06-23|1      |5942500         |1   |2  |27    |89138       |5      |9044        |129    |2020-06-20|113    |5058000    |2     |
+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+


Vi kan skriva SQL-queries:

In [12]:
result = spark.sql("SELECT * FROM personer LIMIT 5")
result.show()

+------+---------+--------------------+-------+---+----------+----------+
|adress|efternamn|               email|fornamn| id|  personnr| telefonnr|
+------+---------+--------------------+-------+---+----------+----------+
|   102| Wudeland|mwudeland0@drupal...|Danièle|  1|1997-12-19|3114705221|
|   104|   Prayer|jprayer1@wordpres...|    Zoé|  2|1958-02-26|6162426013|
|   106|  Dimmick|   hdimmick2@epa.gov|Thérèse|  3|1988-07-25|5095529405|
|   108| Cuddehay|scuddehay3@delici...| Noémie|  4|1962-08-23|3451506020|
|   110|   Cheley|ycheley4@shop-pro.jp|  André|  5|1976-12-26|6175439945|
+------+---------+--------------------+-------+---+----------+----------+


Våra queries kan göra joins mellan temp views (eller tabeller):

In [13]:
result = spark.sql("""
    SELECT * FROM objekt, personer 
        WHERE objekt.saljare = personer.id 
        LIMIT 5
""")
result.show()

+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+------+-----------+--------------------+----------+---+----------+----------+
|adress|antal_rum|  avslutat|balkong|forsaljningspris|hiss| id|kopare|maklararvode|maklare|manadsavgift|saljare| saljstart|storlek|utgangspris|vaning|adress|  efternamn|               email|   fornamn| id|  personnr| telefonnr|
+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+------+-----------+--------------------+----------+---+----------+----------+
|    75|        6|2020-07-19|      1|        14561000|   1|  1|     5|      218415|      2|        3090|    107|2020-01-25|    137|   12420000|     5|   153|Osbaldstone|eosbaldstone2y@ml...|Marie-noël|107|1976-05-11|3178324649|
|    72|        5|2020-06-23|      1|         5942500|   1|  2|    27|       89138|     

Vi kan skapa mer komplexa frågor, med UNIONs:
C. Vilken storlek på lägenheter säljer snabbast i Skåne? De som är under 50kvm, mellan 50-100kvm eller över 100kvm?

In [14]:
result = spark.sql("""
SELECT "Mindre än 50 kvm" AS storlek, ceil(avg(datediff(avslutat, saljstart))) AS dagar FROM objekt WHERE avslutat IS NOT NULL 
AND storlek < 50
UNION
SELECT "Mellan 50–100 kvm" AS storlek, ceil(avg(datediff(avslutat, saljstart))) AS dagar FROM objekt WHERE avslutat IS NOT NULL 
AND storlek BETWEEN 50 AND 100
UNION
SELECT "Större än 50 kvm" AS storlek, ceil(avg(datediff(avslutat, saljstart))) AS dagar FROM objekt WHERE avslutat IS NOT NULL 
AND storlek > 100
ORDER BY dagar LIMIT 1
""") 
result.show()

+-----------------+-----+
|          storlek|dagar|
+-----------------+-----+
|Mellan 50–100 kvm|  116|
+-----------------+-----+


Men om jag vill kunna skapa VIEWs behöver jag kunna göra queries mot permanenta tabeller. Och om jag vill spara egna permanenta SQL-tabeller eller spara dataframes permanent för SQL-querying, behöver jag konvertera mina dataframes (eller temp views) till tabeller:

In [15]:
# Skapa en "hive table" (permanent tabell). 
# OBS! Notera att namnet på tabellen inte kan vara samma som min temp view: 
spark.sql("CREATE TABLE objekt_table AS SELECT * FROM objekt")

24/02/21 15:37:05 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
24/02/21 15:37:05 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/02/21 15:37:05 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/02/21 15:37:07 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/02/21 15:37:07 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore b@10.200.51.175
24/02/21 15:37:07 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
24/02/21 15:37:07 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAut

DataFrame[]

_(Notera att operationerna med att skapa hive-tabeller ger rätt mycket varningar, men fungerar ändå som de ska)_

Jag kan däremot ersätta min ursprungliga dataframe med en permanent tabell. Så här skulle det kunna se ut:

In [16]:
# Läser in en JSON-fil till en dataframe
bostadsratter = spark.read.json("spark-warehouse/bostadsratter.json", multiLine=True)

# Skapa en temp-view
bostadsratter.createOrReplaceTempView("bostadsratter_temp_view")

# Skapa en hive-tabell från min temp-view
spark.sql("CREATE TABLE bostadsratter AS SELECT * FROM bostadsratter_temp_view")

24/02/21 15:37:08 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
24/02/21 15:37:08 WARN HiveMetaStore: Location: file:/Users/b/Desktop/Projects/DM23/Homenet/spark-warehouse/bostadsratter specified for non-external table:bostadsratter


DataFrame[]

In [18]:
result = spark.sql("SELECT * FROM bostadsratter")
result.show(3,0)

+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+
|adress|antal_rum|avslutat  |balkong|forsaljningspris|hiss|id |kopare|maklararvode|maklare|manadsavgift|saljare|saljstart |storlek|utgangspris|vaning|
+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+
|75    |6        |2020-07-19|1      |14561000        |1   |1  |5     |218415      |2      |3090        |107    |2020-01-25|137    |12420000   |5     |
|72    |5        |2020-06-23|1      |5942500         |1   |2  |27    |89138       |5      |9044        |129    |2020-06-20|113    |5058000    |2     |
|72    |3        |NULL      |0      |NULL            |0   |3  |NULL  |NULL        |1      |7075        |79     |2020-09-24|126    |11194000   |2     |
+------+---------+----------+-------+----------------+----+---+------+------------+-------+---

Nu kan vi skapa VIEWs:

In [19]:
spark.sql("CREATE OR REPLACE VIEW bostadsratter_till_forsaljning AS SELECT * FROM bostadsratter WHERE avslutat IS NOT NULL")

DataFrame[]

In [22]:
result = spark.sql("SELECT * FROM bostadsratter_till_forsaljning")
result.show(3,0)

+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+
|adress|antal_rum|avslutat  |balkong|forsaljningspris|hiss|id |kopare|maklararvode|maklare|manadsavgift|saljare|saljstart |storlek|utgangspris|vaning|
+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+
|75    |6        |2020-07-19|1      |14561000        |1   |1  |5     |218415      |2      |3090        |107    |2020-01-25|137    |12420000   |5     |
|72    |5        |2020-06-23|1      |5942500         |1   |2  |27    |89138       |5      |9044        |129    |2020-06-20|113    |5058000    |2     |
|24    |1        |2020-10-23|1      |1563000         |1   |4  |150   |23445       |6      |6033        |21     |2020-02-08|40     |1332000    |1     |
+------+---------+----------+-------+----------------+----+---+------+------------+-------+---